In [4]:
%matplotlib inline

import json
import pandas as pd
import numpy as np

In [2]:
all_matches = pd.read_csv('all_t20i_05-16.csv')
all_matches = all_matches.sort('date', ascending = False)
matches = all_matches.match_id.values

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [3]:
def f(i):
    return i['bat'], i['bowl'], i['o_u'], i['ovr'], i['r'], i['r_t'], i['x'], i['y'], i['z']

In [6]:
import os.path
failed_ids = []

for idx, match_id in enumerate(matches):
    try:
        wicket_data = json.load(open('data/%s-pie-wickets.json'%(match_id)))
        df = pd.io.json.read_json('data/%s-wagon-inning-1.json'%(match_id), dtype=False)    
        df['inning'] = 1
        df['batting_team'] = wicket_data['t1']['n']
        df['bowling_team'] = wicket_data['t2']['n']

        df2 = pd.io.json.read_json('data/%s-wagon-inning-2.json'%(match_id), dtype=False)
        df2['inning'] = 2
        df2['batting_team'] = wicket_data['t2']['n']
        df2['bowling_team'] = wicket_data['t1']['n']

        df['batsman'], df['bowler'], df['ball_num'], df['ovr'], df['runs_batter'], df['runs_w_extras'], df['x'], df['y'], df['z'] = zip(*df.runs.apply(f))
        df2['batsman'], df2['bowler'], df2['ball_num'], df2['ovr'], df2['runs_batter'], df2['runs_w_extras'], df2['x'], df2['y'], df2['z'] = zip(*df2.runs.apply(f))

        for param in ['ball_num', 'ovr', 'runs_batter', 'runs_w_extras', 'x', 'y', 'z']:
            df[param] = df[param].astype(float)
            df2[param] = df2[param].astype(float)

        from collections import defaultdict
        dismissals = defaultdict(dict)

        for i in wicket_data['t1']['w']:
            dismissals[float(i['ovr'])] = {'inning': 1, 'how': i['how'], 'ovr': float(i['ovr']), 'batsman': i['out'], 'bowler': i['outby']}

        for i in wicket_data['t2']['w']:
            dismissals[float(i['ovr'])] = {'inning': 2, 'how': i['how'], 'ovr': float(i['ovr']), 'batsman': i['out'], 'bowler': i['outby']}

        def add_wickets(inning, ovr, batsman, extras, unique, dismissals = dismissals):
            #Note: a run-out on a no-ball or wide will be double-counted
            if ovr in dismissals and dismissals[ovr]['inning'] == inning:
                if unique == 'True':
                    return True, dismissals[ovr]['how'], dismissals[ovr]['batsman']
                else:
                    if dismissals[ovr]['how'] != 'run out':
                        if extras == 0:
                            return True, dismissals[ovr]['how'], dismissals[ovr]['batsman']
                        else:
                            return False, np.nan, np.nan
                    else:
                        return True, dismissals[ovr]['how'], dismissals[ovr]['batsman'] #think of a better way later
            return False, np.nan, np.nan

        val_count_1 = df.ovr.value_counts()
        val_count_2 = df2.ovr.value_counts()

        def get_unique_1(i):
            if val_count_1[i] == 1:
                return True
            return False

        def get_unique_2(i):
            if val_count_2[i] == 1:
                return True
            return False

        df['ovr_unique'] = df.ovr.apply(get_unique_1)
        df2['ovr_unique'] = df2.ovr.apply(get_unique_2)
        df['extras'] = df['runs_w_extras'] - df['runs_batter']
        df2['extras'] = df2['runs_w_extras'] - df2['runs_batter']
        df['wicket'], df['wicket_method'], df['who_out'] = zip(*df.apply(lambda row: add_wickets(row['inning'], row['ovr'], row['batsman'], row['extras'], row['ovr_unique']), axis=1))
        df2['wicket'], df2['wicket_method'], df2['who_out'] = zip(*df2.apply(lambda row: add_wickets(row['inning'], row['ovr'], row['batsman'], row['extras'], row['ovr_unique']), axis=1))
        df['match_id'] = match_id
        df2['match_id'] = match_id

        df['cumul_runs'] = df['runs_w_extras'].cumsum()
        df['cumul_wickets'] = df['wicket'].cumsum()
        df['cumul_balls'] = df['ovr'].apply(lambda x: 6*int(x) + int(10*x)%10)
        df2['cumul_runs'] = df2['runs_w_extras'].cumsum()
        df2['cumul_wickets'] = df2['wicket'].cumsum()
        df2['cumul_balls'] = df2['ovr'].apply(lambda x: 6*int(x) + int(10*x)%10)
        
        player_db = {}
        inn_1_batting_order = {}
        inn_2_batting_order = {}
        for idx__, d in enumerate(wicket_data['t1']['p']):
            player_db.update(d)
            inn_1_batting_order[d.keys()[0]] = idx__+1
        for idx__, d in enumerate(wicket_data['t2']['p']):
            player_db.update(d)
            inn_2_batting_order[d.keys()[0]] = idx__+1

        def get_name(i):
            return player_db.get(i, None)

        df['batsman_name'] = df['batsman'].apply(get_name)
        df['bowler_name'] = df['bowler'].apply(get_name)
        df['who_out'] = df['who_out'].apply(get_name)
        df['batting_order'] = df['batsman'].apply(inn_1_batting_order.get)
        
        df2['batsman_name'] = df2['batsman'].apply(get_name)
        df2['bowler_name'] = df2['bowler'].apply(get_name)
        df2['who_out'] = df2['who_out'].apply(get_name)
        df2['batting_order'] = df2['batsman'].apply(inn_2_batting_order.get)
        
        df = df.append(df2)    
        df = df.drop('runs', axis=1)
        df = df.reset_index(drop=True)

        if os.path.isfile('data/%s_ball_details.txt'%(match_id)):
            try:
                with open('data/%s_ball_details.txt'%(match_id), 'r') as fil:
                    a = eval(fil.read()[8:-2])
                l = []
                for i in a[0]['data']:
                    temp = i.values()[0].split(',')
                    l.append({'ball_speed': temp[3], 'landing_y': temp[4], 'landing_x': temp[5], 'bat_right_handed': temp[6],
                         'ended_x': temp[7], 'ended_y': temp[8]})
                df2 = pd.DataFrame(l)
                if len(df2) == len(df):
                    df = df.join(df2)
                    cols = ['inning', 'batting_team', 'bowling_team', 'batsman', 'bowler', 'batsman_name', 'batting_order',
                        'bowler_name', 'bat_right_handed', 'ovr', 'runs_batter', 'runs_w_extras', 'extras',
                        'x', 'y', 'z', 'landing_x', 'landing_y', 'ended_x', 'ended_y', 'ball_speed', 'cumul_runs',
                        'cumul_wickets', 'cumul_balls', 'wicket', 'wicket_method', 'who_out']
                else:
                    print 'discrepancy', match_id
                    cols = ['inning', 'batting_team', 'bowling_team', 'batsman', 'bowler', 'batsman_name', 'batting_order',
                    'bowler_name', 'ovr', 'runs_batter', 'runs_w_extras', 'extras',
                    'x', 'y', 'z', 'cumul_runs', 'cumul_wickets', 'cumul_balls', 'wicket', 'wicket_method', 'who_out']
            except:
                cols = ['inning', 'batting_team', 'bowling_team', 'batsman', 'bowler', 'batsman_name', 'batting_order',
                    'bowler_name', 'ovr', 'runs_batter', 'runs_w_extras', 'extras',
                    'x', 'y', 'z', 'cumul_runs', 'cumul_wickets', 'cumul_balls', 'wicket', 'wicket_method', 'who_out']
        df = df[cols]
        df.to_csv('cleaned_data/%s_summary.csv'%(match_id))
        print 'success', idx, len(matches)
    except:
        print 'failed', idx, match_id, len(matches)
        failed_ids.append(int(match_id))

success 0 505
success 1 505
success 2 505
success 3 505
success 4 505
success 5 505
success 6 505
success 7 505
success 8 505
success 9 505
success 10 505
success 11 505
success 12 505
success 13 505
success 14 505
success 15 505
success 16 505
success 17 505
success 18 505
success 19 505
success 20 505
success 21 505
success 22 505
success 23 505
success 24 505
success 25 505
success 26 505
success 27 505
success 28 505
success 29 505
success 30 505
success 31 505
success 32 505
success 33 505
success 34 505
success 35 505
success 36 505
success 37 505
success 38 505
success 39 505
success 40 505
success 41 505
success 42 505
success 43 505
success 44 505
success 45 505
success 46 505
success 47 505
success 48 505
success 49 505
success 50 505
success 51 505
success 52 505
success 53 505
success 54 505
success 55 505
success 56 505
success 57 505
success 58 505
success 59 505
success 60 505
success 61 505
success 62 505
success 63 505
success 64 505
success 65 505
success 66 505
succe

### List of matches that were not parsed

In [7]:
all_matches[all_matches.match_id.isin(failed_ids)]

,Unnamed: 0,team1_name,team2_name,team1_id,team2_id,ground_name,ground_id,match_id,date
326,326,Afghanistan,Netherlands,40,15,Sharjah,59392,660107,2013-11-15
8,8,Bangladesh,Zimbabwe,25,9,Khulna,56691,264065,2006-11-28
